In [3]:
import nltk
from nltk.corpus import sentence_polarity
from nltk.corpus import names
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
import pandas as pd
import random
import re
from sklearn import svm
from sklearn.metrics import classification_report

In [34]:
tweeter_init = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/olid-training-v1.csv") 

In [114]:
tweeter_init[:15]

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN
7,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
8,45157,@USER Buy more icecream!!!,NOT,NaN,NaN
9,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND


In [36]:
testa = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/testset-levela.tsv", sep = '\t') 
labela = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/labels-levela.csv", header = None)

In [37]:
testa[:5]

,id,tweet
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1,27014,"#ConstitutionDay is revered by Conservatives, ..."
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3,13876,#Watching #Boomer getting the news that she is...
4,60133,#NoPasaran: Unity demo to oppose the far-right...


In [38]:
labela[:5]

,0,1
0,15923,OFF
1,27014,NOT
2,30530,NOT
3,13876,NOT
4,60133,OFF


In [39]:
labela.columns = ['id1', 'subtask_a']

In [40]:
testa = pd.merge(testa, labela, on=testa.index, how='inner').drop('id1', axis=1).drop('key_0', axis=1)


In [41]:
testa[:5]

,id,tweet,subtask_a
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF


In [42]:
len(testa)

860

In [43]:
testb = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/testset-levelb.tsv", sep = '\t') 
labelb = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/labels-levelb.csv", header = None)

In [44]:
labelb.columns = ['id1', 'subtask_b']

In [45]:
testb = pd.merge(testb, labelb, on=testb.index, how='inner').drop('id1', axis=1).drop('key_0', axis=1)

In [46]:
testb[:5]

,id,tweet,subtask_b
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,TIN
1,60133,#NoPasaran: Unity demo to oppose the far-right...,TIN
2,83681,. . . What the fuck did he do this time?,TIN
3,65507,@USER Do you get the feeling he is kissing @US...,TIN
4,12588,@USER Nigga ware da hits at,UNT


In [47]:
testc = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/testset-levelc.tsv", sep = '\t') 
labelc = pd.read_csv("/Users/xinxiasong/Downloads/IST664/project/OLIDv1.0/labels-levelc.csv", header = None)

In [48]:
labelc.columns = ['id1', 'subtask_c']

In [49]:
testc = pd.merge(testc, labelc, on=testc.index, how='inner').drop('id1', axis=1).drop('key_0', axis=1)

In [50]:
testc[:5]

,id,tweet,subtask_c
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OTH
1,60133,#NoPasaran: Unity demo to oppose the far-right...,GRP
2,83681,. . . What the fuck did he do this time?,IND
3,65507,@USER Do you get the feeling he is kissing @US...,OTH
4,34263,#StopKavanaugh he is liar like the rest of the...,IND


In [51]:
# Method 1: featuresets
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)

In [52]:
documents =[]
for i in range(len(tweeter_init['id'])):
    tw0 = tknzr.tokenize(tweeter_init['tweet'][i])
    tw1 = [w.lower() for w in tw0]      
    documents.append(((tw1), \
                     tweeter_init['subtask_a'][i], \
                     tweeter_init['subtask_b'][i], \
                     tweeter_init['subtask_c'][i]))


In [53]:
documents[:1]

[(['@user',
   'she',
   'should',
   'ask',
   'a',
   'few',
   'native',
   'americans',
   'what',
   'their',
   'take',
   'on',
   'this',
   'is',
   '.'],
  'OFF',
  'UNT',
  nan)]

In [54]:
random.shuffle(documents)

In [55]:
all_words_list = [word for (sent, a, b, c) in documents for word in sent]

In [56]:
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather',\
                 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

In [57]:
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

newstopwords = [word for word in stopwords if word not in negationwords and not (word.endswith("n't"))]

non_alpha = ['.',',',':','"','-','&','(',')','/','”','“','’',"'",'*','u',';','—']
for item in non_alpha:
    newstopwords.append(item)
print(newstopwords)

all_words_list = [word for word in all_words_list if word not in newstopwords]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(2500)
word_features = [word for (word,count) in word_items]
word_features[200]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

'looks'

In [58]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [59]:
featurea = [(document_features(d,word_features), a) for (d, a, b, c) in documents]

In [60]:
print(len(featurea))

13240


In [100]:
featureb = [(document_features(d,word_features), b) for (d, a, b, c) in documents if a != 'NOT']

In [115]:
featurec = [(document_features(d,word_features), c) for (d, a, b, c) in documents if b == 'TIN']

In [116]:
featurec[1]

({'contains(@user)': True,
  'contains(!)': False,
  'contains(?)': False,
  'contains(url)': True,
  'contains(...)': False,
  'contains(not)': False,
  'contains(liberals)': False,
  'contains(gun)': False,
  'contains(control)': False,
  'contains(like)': False,
  'contains(antifa)': False,
  'contains(people)': False,
  'contains(conservatives)': False,
  'contains(no)': False,
  'contains(#maga)': True,
  'contains(get)': False,
  'contains(know)': False,
  'contains(one)': False,
  'contains(trump)': False,
  'contains(think)': False,
  'contains(would)': False,
  "contains(don't)": False,
  'contains(😂)': False,
  'contains(right)': False,
  'contains(good)': False,
  'contains(..)': False,
  'contains(us)': False,
  'contains(time)': False,
  'contains(want)': False,
  'contains(go)': False,
  'contains(🇺)': False,
  'contains(🇸)': False,
  'contains(see)': False,
  'contains(need)': False,
  'contains(never)': False,
  'contains(going)': False,
  'contains(shit)': False,
  'co

In [117]:
train_a = featurea
train_b = featureb
train_c = featurec

In [64]:
classifier1a = nltk.NaiveBayesClassifier.train(train_a)

In [103]:
classifier1b = nltk.NaiveBayesClassifier.train(train_b)

In [118]:
classifier1c = nltk.NaiveBayesClassifier.train(train_c)

In [67]:
testa_doc =[]
for i in range(len(testa['id'])):
    tw0 = tknzr.tokenize(testa['tweet'][i])
    tw1 = [w.lower() for w in tw0] 
    tw2 = [w for w in tw1 if w not in newstopwords]
    testa_doc.append(((tw2), testa['subtask_a'][i]))

In [68]:
testb_doc =[]
for i in range(len(testb['id'])):
    tw0 = tknzr.tokenize(testb['tweet'][i])
    tw1 = [w.lower() for w in tw0] 
    tw2 = [w for w in tw1 if w not in newstopwords]
    testb_doc.append(((tw2), testb['subtask_b'][i]))

In [69]:
testc_doc =[]
for i in range(len(testc['id'])):
    tw0 = tknzr.tokenize(testc['tweet'][i])
    tw1 = [w.lower() for w in tw0] 
    tw2 = [w for w in tw1 if w not in newstopwords]
    testc_doc.append(((tw2), testc['subtask_c'][i]))

In [70]:
testa_sets = [(document_features(d,word_features), a) for (d, a) in testa_doc]

In [71]:
testb_sets = [(document_features(d,word_features), b) for (d, b) in testb_doc]

In [72]:
testc_sets = [(document_features(d,word_features), c) for (d, c) in testc_doc]

In [73]:
test_a = testa_sets
test_b = testb_sets
test_c = testc_sets

In [74]:
print(nltk.classify.accuracy(classifier1a, test_a))

0.7976744186046512


In [105]:
print(nltk.classify.accuracy(classifier1b, test_b))

0.8375


In [123]:
print(nltk.classify.accuracy(classifier1c, test_c))

0.5117370892018779


In [77]:
import nltk.classify
from sklearn.svm import LinearSVC

In [78]:
classifier1a_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier1a_svm.train(train_a)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [79]:
print(nltk.classify.accuracy(classifier1a_svm, test_a))

0.7895348837209303


In [125]:
classifier1b_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier1b_svm.train(train_b)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [126]:
print(nltk.classify.accuracy(classifier1b_svm, test_b))

0.8083333333333333


In [127]:
classifier1c_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier1c_svm.train(train_c)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [128]:
print(nltk.classify.accuracy(classifier1c_svm, test_c))

0.5211267605633803


In [34]:
# Method 2 NOT featuresets

In [84]:
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = False
        features['contains(NOT{})'.format(word)] = False
         # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
        else:
            features['contains({})'.format(word)] = (word in word_features)
    return features

In [85]:
NOT_featurea = [(NOT_features(d, word_features, negationwords), a) for (d, a, b, c) in documents]
len(NOT_featurea)

13240

In [129]:
NOT_featureb = [(NOT_features(d, word_features, negationwords), b) for (d, a, b, c) in documents if a == 'OFF']
len(NOT_featureb)

4400

In [157]:
NOT_featureb[0]

({'contains(@user)': True,
  'contains(NOT@user)': False,
  'contains(!)': False,
  'contains(NOT!)': False,
  'contains(?)': False,
  'contains(NOT?)': False,
  'contains(url)': False,
  'contains(NOTurl)': False,
  'contains(...)': False,
  'contains(NOT...)': False,
  'contains(not)': False,
  'contains(NOTnot)': False,
  'contains(liberals)': False,
  'contains(NOTliberals)': False,
  'contains(gun)': False,
  'contains(NOTgun)': False,
  'contains(control)': False,
  'contains(NOTcontrol)': False,
  'contains(like)': False,
  'contains(NOTlike)': False,
  'contains(antifa)': False,
  'contains(NOTantifa)': False,
  'contains(people)': False,
  'contains(NOTpeople)': False,
  'contains(conservatives)': False,
  'contains(NOTconservatives)': False,
  'contains(no)': False,
  'contains(NOTno)': False,
  'contains(#maga)': False,
  'contains(NOT#maga)': False,
  'contains(get)': True,
  'contains(NOTget)': False,
  'contains(know)': False,
  'contains(NOTknow)': False,
  'contains(one

In [130]:
NOT_featurec = [(NOT_features(d, word_features, negationwords), c) for (d, a, b, c) in documents if b == 'TIN']
len(NOT_featurec)

3876

In [158]:
NOT_featurec[0]

({'contains(@user)': True,
  'contains(NOT@user)': False,
  'contains(!)': False,
  'contains(NOT!)': False,
  'contains(?)': False,
  'contains(NOT?)': False,
  'contains(url)': False,
  'contains(NOTurl)': False,
  'contains(...)': False,
  'contains(NOT...)': False,
  'contains(not)': False,
  'contains(NOTnot)': False,
  'contains(liberals)': False,
  'contains(NOTliberals)': False,
  'contains(gun)': False,
  'contains(NOTgun)': False,
  'contains(control)': False,
  'contains(NOTcontrol)': False,
  'contains(like)': False,
  'contains(NOTlike)': False,
  'contains(antifa)': False,
  'contains(NOTantifa)': False,
  'contains(people)': False,
  'contains(NOTpeople)': False,
  'contains(conservatives)': False,
  'contains(NOTconservatives)': False,
  'contains(no)': False,
  'contains(NOTno)': False,
  'contains(#maga)': False,
  'contains(NOT#maga)': False,
  'contains(get)': True,
  'contains(NOTget)': False,
  'contains(know)': False,
  'contains(NOTknow)': False,
  'contains(one

In [147]:
train_a2 = NOT_featurea

In [148]:
train_b2 = NOT_featureb

In [149]:
train_c2 = NOT_featurec

In [150]:
classifier2a = nltk.NaiveBayesClassifier.train(train_a2)

In [151]:
classifier2b = nltk.NaiveBayesClassifier.train(train_b2)

In [152]:
classifier2c = nltk.NaiveBayesClassifier.train(train_c2)

In [140]:
NOT_testa = [(NOT_features(d, word_features, negationwords), a) for (d, a) in testa_doc]

In [161]:
NOT_testb = [(NOT_features(d, word_features, negationwords), b) for (d, b) in testb_doc]

In [160]:
NOT_testb[0]

({'contains(@user)': True,
  'contains(NOT@user)': False,
  'contains(!)': True,
  'contains(NOT!)': False,
  'contains(?)': False,
  'contains(NOT?)': False,
  'contains(url)': True,
  'contains(NOTurl)': False,
  'contains(...)': False,
  'contains(NOT...)': False,
  'contains(not)': False,
  'contains(NOTnot)': False,
  'contains(liberals)': False,
  'contains(NOTliberals)': False,
  'contains(gun)': False,
  'contains(NOTgun)': False,
  'contains(control)': False,
  'contains(NOTcontrol)': False,
  'contains(like)': False,
  'contains(NOTlike)': False,
  'contains(antifa)': True,
  'contains(NOTantifa)': False,
  'contains(people)': False,
  'contains(NOTpeople)': False,
  'contains(conservatives)': False,
  'contains(NOTconservatives)': False,
  'contains(no)': False,
  'contains(NOTno)': False,
  'contains(#maga)': False,
  'contains(NOT#maga)': False,
  'contains(get)': False,
  'contains(NOTget)': False,
  'contains(know)': False,
  'contains(NOTknow)': False,
  'contains(one)'

In [162]:
NOT_testc = [(NOT_features(d, word_features, negationwords), c) for (d, c) in testc_doc]

In [141]:
test_NOT_a = NOT_testa

In [163]:
test_NOT_b = NOT_testb

In [164]:
test_NOT_c = NOT_testc

In [153]:
print(nltk.classify.accuracy(classifier2a, test_NOT_a))

0.7418604651162791


In [165]:
print(nltk.classify.accuracy(classifier2b, test_NOT_b))

0.7333333333333333


In [166]:
print(nltk.classify.accuracy(classifier2c, test_NOT_c))

0.49765258215962443


In [167]:
classifier2a_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier2a_svm.train(train_a2)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [168]:
print(nltk.classify.accuracy(classifier2a_svm, test_NOT_a))

0.7767441860465116


In [169]:
classifier2b_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier2b_svm.train(train_b2)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [170]:
print(nltk.classify.accuracy(classifier2b_svm, test_NOT_b))

0.7916666666666666


In [171]:
classifier2c_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier2c_svm.train(train_c2)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [172]:
print(nltk.classify.accuracy(classifier2c_svm, test_NOT_c))

0.5539906103286385
